PSN

In [1]:
from psnawp_api import PSNAWP
from tqdm import tqdm
import pandas as pd
import numpy as np

# Get your npsso from https://psnprofiles.com/psnawp
npsso = "lu6TxZVE8LwVB7DfEh4DBzDlHd80DsubdpYpr4LV1eACfwo35RFiZTyOeA2MxPhO"
# Create a PSNAWP object
psn = PSNAWP(npsso)

# Profile of npsso owner
client = psn.me()
print("account_Id : " + client.account_id + ", online_id : " + client.online_id)
print("totTrophies : " + str(client.trophy_summary().earned_trophies.bronze + client.trophy_summary().earned_trophies.silver + client.trophy_summary().earned_trophies.gold + client.trophy_summary().earned_trophies.platinum))

listOfList = []



#gameCount = 0
# for t in client.title_stats():
#    print("name : " + t.name + " , title_id : " + str(t.title_id) + " , play_count : " + str(t.play_count) + " ,total_items_count : " + str(t.total_items_count))
#    gameCount += 1

trophyCount = 0
for tr in tqdm(client.trophy_titles()):
    listGame = [tr.np_communication_id, tr.title_name,
                (tr.defined_trophies.bronze + tr.defined_trophies.silver + tr.defined_trophies.gold + tr.defined_trophies.platinum),
                (tr.earned_trophies.bronze + tr.earned_trophies.silver + tr.earned_trophies.gold + tr.earned_trophies.platinum),
                (str(tr.progress) + "%")]
    listOfList.append(listGame)
    trophyCount += 1

arr = np.array(listOfList)
df = pd.DataFrame(arr, columns=["np_communication_id", "title_name", "totTrophy", "earnedTrophy", "percTrophy"])
#print("Game Count : " + str(gameCount))
print("Game Count : " + str(trophyCount))
print(df)



account_Id : 3952657318490276191, online_id : AterMoros
totTrophies : 7802


648it [00:06, 101.04it/s]

Game Count : 648
    np_communication_id             title_name totTrophy earnedTrophy  \
0          NPWR30769_00          SILENT HILL 2        44            4   
1          NPWR34665_00               TEKKEN 8        47            4   
2          NPWR15473_00  Stick Fight: The Game        28            7   
3          NPWR20652_00           Helldivers 2        39           12   
4          NPWR22032_00          DOOM Eternal®        51           27   
..                  ...                    ...       ...          ...   
643        NPWR00518_00      The Godfather™ II        42           35   
644        NPWR00650_00             L.A. Noire        61           12   
645        NPWR01768_00      SONIC GENERATIONS        50           26   
646        NPWR01500_00          DCUO Trophies       123            0   
647        NPWR00464_00            Dead Space™        49            0   

    percTrophy  
0           5%  
1           5%  
2          12%  
3          18%  
4          54%  
..  

Steam

In [4]:
from steam_web_api import Steam
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from dotenv import load_dotenv

load_dotenv()

KEY = os.getenv("STEAM_API_KEY")
steam = Steam(KEY)

user = steam.users.get_user_details("76561198074617013")
# print(user)
# print(user["player"])
# print("PLAYER DETAILS")
# for i in user["player"]:
# print(str(i) + " : " + str(user["player"][i]))

print(
    "STEAMID: "
    + str(user["player"]["steamid"])
    + " NAME : "
    + str(user["player"]["personaname"])
)


# friends = steam.users.get_user_friends_list("76561198074617013")
# print(friends)

steamId = str(user["player"]["steamid"])  # 76561198074617013

listOfList = []


# GAMES
games = steam.users.get_owned_games(steamId)
print("Game Count : " + str(games.get("game_count")))
print("Recupero Dati : ")
for i in tqdm(games.get("games")):
    listGame = [i.get("appid"), i.get("name"), i.get("playtime_forever")]
    # print("appId: " + str(i.get("appid")) + " / Name: " + str(i.get("name")) + " / Playtime: " + str(i.get("playtime_forever")))
    try:
        if i.get("playtime_forever") > 0:
            gameS = steam.apps.get_user_stats(steamId, i.get("appid"))
            g = steam.apps.get_app_details(i.get("appid"))
            totAchievement = int(
                g.split("achievements")[1].split("total")[1].split(",")[0].split(" ")[1]
            )
            earnedAchievement = len(gameS.get("playerstats").get("achievements"))
            # print("Tot Achievement : " + str(totAchievement))
            listGame.append(str(totAchievement))
            # print("Earned Achievement : " + str(earnedAchievement))
            listGame.append(str(earnedAchievement))
            if totAchievement > 0:
                percAchievement = (earnedAchievement / totAchievement) * 100
                # print("Achievement % : " + str(round(percAchievement, 2)) + "%")
                listGame.append(str(round(percAchievement, 2)) + "%")
            else:
                # print("Achievement % : 0 %")
                listGame.append(str(0) + "%")
        else:
            # print("No Achievement Data")
            listGame.append(str(0))
            listGame.append(str(0))
            listGame.append(str(0) + "%")
    except:
        # print("No Achievement Data")
        listGame.append(str(0))
        listGame.append(str(0))
        listGame.append(str(0) + "%")
    listOfList.append(listGame)

# TODO : organizzare dati in dataFrame

arr = np.array(listOfList)
df = pd.DataFrame(
    arr,
    columns=[
        "appId",
        "Name",
        "Playtime",
        "TotAchievement",
        "EarnedAchievement",
        "Achievement%",
    ],
)
print(df)


searchGame = steam.apps.search_games("Dota 2")
# print(searchGame)

gameAchievement = steam.apps.get_user_achievements("76561198074617013", 220)
# print(gameAchievement)

gameStats = steam.apps.get_user_stats("76561198074617013", 220)
# print(gameStats)

gameDetail = steam.apps.get_app_details(220)
# print(gameDetail)


STEAMID: 76561198074617013 NAME : AterMors
Game Count : 286
Recupero Dati : 


100%|██████████| 286/286 [03:39<00:00,  1.30it/s]


      appId                                    Name Playtime TotAchievement  \
0       220                             Half-Life 2      266              0   
1       320                 Half-Life 2: Deathmatch        0              0   
2       360            Half-Life Deathmatch: Source        0              0   
3      4000                             Garry's Mod        5              0   
4      6910       Deus Ex: Game of the Year Edition       22              0   
..      ...                                     ...      ...            ...   
281  383180  Dead Island Riptide Definitive Edition        0              0   
282  235900                            RPG Maker XP        0              0   
283     440                         Team Fortress 2      176              0   
284     570                                  Dota 2      117              0   
285    1840                        Source Filmmaker       20              0   

    EarnedAchievement Achievement%  
0             

GameDB

In [ ]:
import rawgpy

rawg = rawgpy.RAWG("")
